In [1]:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

In [2]:
review_df = spark.read.json(
    path='../data/yelp_dataset_challenge_round9/yelp_academic_dataset_review.json'
).select('user_id', 'business_id', 'stars')

In [3]:
review_df.count()

4153150

In [4]:
review_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)



In [5]:
restaurant_ids_df = spark.read.json(
    path='../data/restaurants'
).select('business_id')

In [6]:
restaurant_review_df = review_df.join(
    other=restaurant_ids_df,
    on='business_id',
    how='inner'
)

In [7]:
restaurant_review_df.count()

2577298

In [8]:
restaurant_review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- stars: long (nullable = true)



In [9]:
restaurant_review_df.write.json(
    path='../data/reviews',
    mode='overwrite',
    compression='gzip'
)

In [12]:
restaurant_review_df.groupBy('user_id').count().count()

721779